In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from optbinning import OptimalBinning
from sklearn.model_selection import train_test_split


from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb


import tensorflow as tf

In [2]:
gn_sub = pd.read_csv("gender_submission.csv")
gn_sub.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [3]:
train = pd.read_csv("train.csv")
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test = pd.read_csv("test.csv")
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## DATA DICTIONARY

### Variable	Definition	Key

survival	Survival	0 = No, 1 = Yes
pclass	Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd
sex	Sex	
Age	Age in years	
sibsp	# of siblings / spouses aboard the Titanic	
parch	# of parents / children aboard the Titanic	
ticket	Ticket number	
fare	Passenger fare	
cabin	Cabin number	
embarked	Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton

In [5]:
train = train[['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Survived']]

In [6]:
def calculate_woe_iv(data, feature, target):
    """
    Calculates the Weight of Evidence (WOE) and Information Value (IV) for a given feature in a dataset
    """
    # Group the data by the feature and calculate the total count and count of each class
    grouped_data = data.groupby(feature)[target].agg(['count', 'sum'])
    grouped_data.columns = ['Total', 'Positive']

    # Calculate the total count and positive count of the target variable
    total_count = grouped_data['Total'].sum()
    total_positive = grouped_data['Positive'].sum()

    # Calculate the percentage of total count and positive count
    grouped_data['Percentage Total'] = grouped_data['Total'] / total_count
    grouped_data['Percentage Positive'] = grouped_data['Positive'] / total_positive

    # Calculate the WOE and IV
    grouped_data['WoE'] = np.log(grouped_data['Percentage Positive'] / grouped_data['Percentage Total'])
    grouped_data['IV'] = (grouped_data['Percentage Positive'] - grouped_data['Percentage Total']) * grouped_data['WoE']

    # Return the WOE and IV values
    return grouped_data['WoE'], grouped_data['IV'].sum()
    print('WOE:\n', woe)
    print('IV:', iv)
    

In [7]:
calculate_woe_iv(train,'Parch','Survived')

C:\Users\khann\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


(Parch
 0   -0.110575
 1    0.361236
 2    0.264387
 3    0.446708
 4        -inf
 5   -0.651904
 6        -inf
 Name: WoE, dtype: float64,
 inf)

In [8]:
train['Parch'].isnull().sum()

0

In [9]:
#Unique Values

def n_unique_value(df):
    d1=list(df.columns)
    d2=df.nunique()
    df_uni=pd.DataFrame({'column_name':d1,'n_unique_value':d2}).reset_index(drop=True)
    return df_uni

n_unique_value(train)

,column_name,n_unique_value
0,PassengerId,891
1,Pclass,3
2,Name,891
3,Sex,2
4,Age,88
5,SibSp,7
6,Parch,7
7,Ticket,681
8,Fare,248
9,Cabin,147


In [10]:
calculate_woe_iv(train,'Cabin','Survived')

C:\Users\khann\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


(Cabin
 A10        -inf
 A14        -inf
 A16    0.405465
 A19        -inf
 A20    0.405465
          ...   
 F33    0.405465
 F38        -inf
 F4     0.405465
 G6    -0.287682
 T          -inf
 Name: WoE, Length: 147, dtype: float64,
 inf)

In [11]:
train['Parch'].value_counts()

0    678
1    118
2     80
5      5
3      5
4      4
6      1
Name: Parch, dtype: int64

In [12]:
train['Cabin'].isnull().sum()

687

### COVERAGE

In [13]:
def find_coverage(df):
    
    c1=list(df.columns)
    c2=df.count() + df.isnull().sum()
    c3=df.isnull().sum()
    c4=c2-c3
    c5=round((c3/c2),2)
    c6=round((c4/c2),2)
    
    df=pd.DataFrame({'column_name':c1,'overall_total_count':c2,'overall_missing_count':c3,'overall_non_missing_count':c4,'overall_missing_pct':c5,'overall_non_missing_pct':c6}).reset_index(drop=True)
    return df

In [14]:
find_coverage(train)

,column_name,overall_total_count,overall_missing_count,overall_non_missing_count,overall_missing_pct,overall_non_missing_pct
0,PassengerId,891,0,891,0.00,1.00
1,Pclass,891,0,891,0.00,1.00
2,Name,891,0,891,0.00,1.00
3,Sex,891,0,891,0.00,1.00
4,Age,891,177,714,0.20,0.80
5,SibSp,891,0,891,0.00,1.00
6,Parch,891,0,891,0.00,1.00
7,Ticket,891,0,891,0.00,1.00
8,Fare,891,0,891,0.00,1.00
9,Cabin,891,687,204,0.77,0.23


### Correlation

In [15]:
def find_correlation(df):
    list_col = df.columns
    cor = df[list_col].corr()
    biz_final_cor = cor.stack().reset_index()
    biz_final_cor.columns = ['column_1','column_2','correlation_coefficient']
    return biz_final_cor

In [16]:
find_correlation(train)

,column_1,column_2,correlation_coefficient
0,PassengerId,PassengerId,1.000000
1,PassengerId,Pclass,-0.035144
2,PassengerId,Age,0.036847
3,PassengerId,SibSp,-0.057527
4,PassengerId,Parch,-0.001652
5,PassengerId,Fare,0.012658
6,PassengerId,Survived,-0.005007
7,Pclass,PassengerId,-0.035144
8,Pclass,Pclass,1.000000
9,Pclass,Age,-0.369226


In [17]:
# One-hot encode "Embarked" column
embarked_onehot = pd.get_dummies(train['Embarked'], prefix='Embarked')
train = pd.concat([train, embarked_onehot], axis=1)

# One-hot encode "Sex" column
sex_onehot = pd.get_dummies(train['Sex'], prefix='Sex')
train = pd.concat([train, sex_onehot], axis=1)

pclass_onehot = pd.get_dummies(train['Pclass'], prefix='Pclass')
train = pd.concat([train, pclass_onehot], axis=1)


# Drop the original "Embarked" and "Sex" columns
train = train.drop(['Embarked', 'Sex','Pclass'], axis=1)

# Print the updated DataFrame
print(train.head())


   PassengerId                                               Name   Age  \
0            1                            Braund, Mr. Owen Harris  22.0   
1            2  Cumings, Mrs. John Bradley (Florence Briggs Th...  38.0   
2            3                             Heikkinen, Miss. Laina  26.0   
3            4       Futrelle, Mrs. Jacques Heath (Lily May Peel)  35.0   
4            5                           Allen, Mr. William Henry  35.0   

   SibSp  Parch            Ticket     Fare Cabin  Survived  Embarked_C  \
0      1      0         A/5 21171   7.2500   NaN         0           0   
1      1      0          PC 17599  71.2833   C85         1           1   
2      0      0  STON/O2. 3101282   7.9250   NaN         1           0   
3      1      0            113803  53.1000  C123         1           0   
4      0      0            373450   8.0500   NaN         0           0   

   Embarked_Q  Embarked_S  Sex_female  Sex_male  Pclass_1  Pclass_2  Pclass_3  
0           0           

In [18]:
#from sklearn.preprocessing import MinMaxScaler

#scaler = MinMaxScaler()
#fare_scaled = scaler.fit_transform(train['Fare'].values.reshape(-1, 1))

# Replace the "fare" column with the scaled values
#train['Fare'] = fare_scaled

In [19]:
train['Age'] = train['Age'].round()
train['Age'].value_counts()

24.0    32
22.0    27
28.0    27
30.0    27
18.0    26
        ..
53.0     1
80.0     1
12.0     1
0.0      1
74.0     1
Name: Age, Length: 71, dtype: int64

In [20]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='most_frequent')
imputer.fit(train[["Age"]])

train[["Age"]] = imputer.transform(train[["Age"]])
      

In [21]:
def calculate_metrics(df, x):
    acc = accuracy_score(df['survived'], df[x])
    prec = precision_score(df['survived'], df[x])
    rec = recall_score(df['survived'], df[x])
    f1 = f1_score(df['survived'], df[x])
    auc = roc_auc_score(df['survived'], df[x])
    
    print(f" Accuracy is {acc}")
    print(f" Precision is {prec:2f}")
    print(f" Recall is {rec:2f}")
    print(f" F1-Score is {f1:2f}")
    print(f" AUC score is {auc:2f}")

In [22]:
target = ['Survived']

In [23]:
train

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Survived,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3
0,1,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,0,0,0,1,0,1,0,0,1
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,1,1,0,0,1,0,1,0,0
2,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,1,0,0,1,1,0,0,0,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,1,0,0,1,1,0,1,0,0
4,5,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,0,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,"Montvila, Rev. Juozas",27.0,0,0,211536,13.0000,NaN,0,0,0,1,0,1,0,1,0
887,888,"Graham, Miss. Margaret Edith",19.0,0,0,112053,30.0000,B42,1,0,0,1,1,0,1,0,0
888,889,"Johnston, Miss. Catherine Helen ""Carrie""",24.0,1,2,W./C. 6607,23.4500,NaN,0,0,0,1,1,0,0,0,1
889,890,"Behr, Mr. Karl Howell",26.0,0,0,111369,30.0000,C148,1,1,0,0,0,1,1,0,0


In [24]:
features = ['Pclass_1','Pclass_2','Pclass_3','Age','SibSp','Parch','Fare','Embarked_C','Embarked_Q','Embarked_S','Sex_female','Sex_male']

In [25]:
train[features]

,Pclass_1,Pclass_2,Pclass_3,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male
0,0,0,1,22.0,1,0,7.2500,0,0,1,0,1
1,1,0,0,38.0,1,0,71.2833,1,0,0,1,0
2,0,0,1,26.0,0,0,7.9250,0,0,1,1,0
3,1,0,0,35.0,1,0,53.1000,0,0,1,1,0
4,0,0,1,35.0,0,0,8.0500,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,1,0,27.0,0,0,13.0000,0,0,1,0,1
887,1,0,0,19.0,0,0,30.0000,0,0,1,1,0
888,0,0,1,24.0,1,2,23.4500,0,0,1,1,0
889,1,0,0,26.0,0,0,30.0000,1,0,0,0,1


In [26]:
X = train[['Pclass_1','Pclass_2','Pclass_3','Age','SibSp','Parch','Fare','Embarked_C','Embarked_Q','Embarked_S','Sex_female','Sex_male']]
y = train['Survived']

In [27]:
X

,Pclass_1,Pclass_2,Pclass_3,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male
0,0,0,1,22.0,1,0,7.2500,0,0,1,0,1
1,1,0,0,38.0,1,0,71.2833,1,0,0,1,0
2,0,0,1,26.0,0,0,7.9250,0,0,1,1,0
3,1,0,0,35.0,1,0,53.1000,0,0,1,1,0
4,0,0,1,35.0,0,0,8.0500,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,1,0,27.0,0,0,13.0000,0,0,1,0,1
887,1,0,0,19.0,0,0,30.0000,0,0,1,1,0
888,0,0,1,24.0,1,2,23.4500,0,0,1,1,0
889,1,0,0,26.0,0,0,30.0000,1,0,0,0,1


In [28]:
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

## SPLITTING X AND Y IN TRAIN AND VAL SET

In [29]:
# Split dataset into training and test sets (80/20 split)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [30]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
X_train = pca.fit_transform(X_train)
X_val = pca.transform(X_val)

In [31]:
X_train.shape

(623, 2)

In [32]:
X_val.shape

(268, 2)

In [36]:
#train2 = X_train.copy()
#train2['survived'] = y_train

#val = X_val.copy()
#val['survived'] = y_val

In [37]:
X_list = ['Pclass_1','Pclass_2','Pclass_3','Age','SibSp','Parch','Fare','Embarked_C','Embarked_Q','Embarked_S','Sex_female','Sex_male']
y_list = ['survived']

In [38]:
X_train

array([[ 49.28487896, -25.65074144],
       [-24.06555141,  -3.47939839],
       [-21.50050225, -26.5970317 ],
       ...,
       [-17.34540512,  13.27446794],
       [ 87.71132649, -16.81690564],
       [ 45.21627376,  -8.51937118]])

In [39]:
X_val

array([[-1.67094532e+01, -3.73525189e+00],
       [-2.12524303e+01,  3.44866277e+00],
       [-2.41547910e+01, -7.47671612e+00],
       [ 5.03141454e-01, -2.22122276e+01],
       [-2.10078895e+01, -1.36038389e+01],
       [ 4.69258971e+01, -3.56563573e+00],
       [-2.42077876e+01, -3.48659288e+00],
       [-1.41957096e+01, -1.18183030e+01],
       [-2.44446391e+01, -1.14795037e+01],
       [-5.81658733e+00, -9.03001515e+00],
       [ 2.15142945e+01,  8.16837437e+00],
       [-2.33192957e+01,  1.64983251e+01],
       [-6.48566423e+00, -4.08829777e+00],
       [-2.45557348e+01,  2.53155173e+00],
       [-1.86055823e+01,  8.37040540e+00],
       [ 7.20180011e+00, -1.24032456e+01],
       [ 2.11167852e+01,  1.31800615e+01],
       [-2.41286251e+01, -3.48893171e+00],
       [-1.88720403e+01, -6.21619319e-01],
       [ 2.07072401e+01,  1.82158017e+01],
       [-2.40655514e+01, -3.47939839e+00],
       [-5.11558789e+00,  5.97887688e+00],
       [-2.42484129e+01, -8.50469233e+00],
       [-2.

## Algorithm

### 1. Logistic Regression with Gridsearch


In [ ]:
from sklearn.model_selection import GridSearchCV

#param_grid = {'penalty': ['l1', 'l2'],
              'C': np.logspace(-4, 4, 20),
              'solver': ['lbfgs', 'liblinear', 'sag', 'newton-cg'],
              'multi_class': ['ovr', 'multinomial'],
              'class_weight': [None, 'balanced'],
              'max_iter': [100, 500, 1000]}

In [ ]:
log_reg = LogisticRegression(C= 100,class_weight = None, max_iter= 100, multi_class='multinomial', penalty='l2', solver='lbfgs')

In [ ]:
#grid_search = GridSearchCV(lg, param_grid, cv=5, n_jobs=-1, verbose=1)

In [ ]:
log_reg.fit(train2[X_list], train2[y_list])

# Print the best hyperparameters
#print(grid_search.best_params_)

In [ ]:
train2['prediction_logreg'] = log_reg.predict(train2[X_list])

In [ ]:
#Train - Conf matrix
cm= confusion_matrix(train2['survived'], train2['prediction_logreg'])
cm

In [ ]:
train2['survived'].value_counts()

In [ ]:
train2['prediction_logreg'].value_counts()

In [ ]:
calculate_metrics(train2,'prediction_logreg')

In [ ]:
val['prediction_logreg'] = log_reg.predict(val[X_list])

In [ ]:
calculate_metrics(val,'prediction_logreg')

In [ ]:
train2.head()

In [ ]:
train2['survived'].value_counts()

### XGboost

In [ ]:
xgb = xgb.XGBClassifier()

In [ ]:
#Training the model using train set

xgb.fit(train2[X_list], train2[y_list])

In [ ]:
# Define the XGBoost model
#xgb_model = xgb.XGBClassifier()

# Define the hyperparameters grid to search
#param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.2, 0.3],
    'n_estimators': [100, 200, 300],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Define the GridSearchCV object
#grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit the GridSearchCV object to the training data
#grid_search.fit(train2[X_list], train2[y_list])

# Print the best hyperparameters and corresponding accuracy score
print("Best hyperparameters:", grid_search.best_params_)
print("Best accuracy score:", grid_search.best_score_)

In [ ]:
#train2['prediction_xgb'] = xgb.predict(train2[X_list])

In [ ]:
#calculate_metrics(train2,'prediction_xgb')

In [ ]:
#val['prediction_xgb'] = xgb.predict(val[X_list])

In [ ]:
#calculate_metrics(val,'prediction_xgb')

### KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=15, weights = 'uniform')

In [ ]:
# train the classifier using the training set
knn.fit(train2[X_list], train2[y_list])

In [ ]:
# make predictions on the Train set
train2['prediction_KNN'] = knn.predict(train2[X_list])

In [ ]:
calculate_metrics(train2,'prediction_KNN')

In [ ]:
val['prediction_KNN'] = knn.predict(val[X_list])

In [ ]:
calculate_metrics(val,'prediction_KNN')

## SVM

In [ ]:
clf = svm.SVC(kernel='rbf', C=1)

In [ ]:
# train the classifier using the training set
clf.fit(train2[X_list], train2[y_list])

In [ ]:
# make predictions on the Train set
train2['prediction_SVM'] = clf.predict(train2[X_list])

In [ ]:
calculate_metrics(train2,'prediction_SVM')

## Decision Tree

In [ ]:
dct = DecisionTreeClassifier(random_state=42)

In [ ]:
# Train the model using the training data
dct.fit(train2[X_list], train2[y_list])

In [ ]:
# make predictions on the Train set
train2['prediction_DT'] = dct.predict(train2[X_list])

In [ ]:
calculate_metrics(train2,'prediction_DT')

In [ ]:
val['prediction_DT'] = dct.predict(val[X_list])

In [ ]:
calculate_metrics(val,'prediction_DT')

## Random Forest Classifier

In [ ]:
rfc = RandomForestClassifier(n_estimators=100)

In [ ]:
# Train the model using the training data
rfc.fit(train2[X_list], train2[y_list])

In [ ]:
# make predictions on the Train set
train2['prediction_RFC'] = rfc.predict(train2[X_list])

In [ ]:
calculate_metrics(train2,'prediction_RFC')

In [ ]:
val['prediction_RFC'] = rfc.predict(val[X_list])

In [ ]:
calculate_metrics(val,'prediction_RFC')

### GBM

In [ ]:
gbm = GradientBoostingClassifier(n_estimators=100, max_depth=3, random_state=42)

In [ ]:
# Train the model using the training data
gbm.fit(train2[X_list], train2[y_list])

In [ ]:
# make predictions on the Train set
train2['prediction_GBM'] = gbm.predict(train2[X_list])

In [ ]:
calculate_metrics(train2,'prediction_GBM')

In [ ]:
# make predictions on the Train set
val['prediction_GBM'] = gbm.predict(val[X_list])

In [ ]:
calculate_metrics(val,'prediction_GBM')

### DOING SAME TRANSFORMATION TECHNIQUES TEST SET

In [ ]:
# One-hot encode "Embarked" column
embarked_onehot = pd.get_dummies(test['Embarked'], prefix='Embarked')
test = pd.concat([test, embarked_onehot], axis=1)

# One-hot encode "Sex" column
sex_onehot = pd.get_dummies(test['Sex'], prefix='Sex')
test = pd.concat([test, sex_onehot], axis=1)

pclass_onehot = pd.get_dummies(test['Pclass'], prefix='Pclass')
test = pd.concat([test, pclass_onehot], axis=1)

# Drop the original "Embarked" and "Sex" columns
test = test.drop(['Embarked', 'Sex'], axis=1)

# Print the updated DataFrame
print(test.head())


In [ ]:
test['Age'] = test['Age'].round()
test['Age'].value_counts()

In [ ]:
test[["Age"]] = imputer.transform(test[["Age"]])

In [ ]:
test[["Fare"]] = imputer.transform(test[["Fare"]])

In [ ]:
#fare_scaled = scaler.transform(test['Fare'].values.reshape(-1, 1))

# Replace the "fare" column with the scaled values
#test['Fare'] = fare_scaled

In [ ]:
test[features]

In [ ]:
test['prediction_logreg'] = log_reg.predict(test[X_list])

In [ ]:
test['prediction_logreg'].value_counts()

In [ ]:
test['prediction_KNN'] = knn.predict(test[X_list])

In [ ]:
test['prediction_KNN'].value_counts()

In [ ]:
titanic_predv1 = test[['PassengerId','prediction_logreg']]

In [ ]:
titanic_predv1 = titanic_predv1.rename(columns={'prediction_logreg':'Survived'})

In [ ]:
titanic_predv1 = titanic_predv1.to_csv("titanic_predv1.csv")

In [ ]:
test['prediction_GBM'] = gbm.predict(test[X_list])

In [ ]:
titanic_predgbm = test[['PassengerId','prediction_GBM']]

In [ ]:
titanic_predgbm = titanic_predgbm.rename(columns={'prediction_GBM':'Survived'})

In [ ]:
titanic_predgbm = titanic_predgbm.to_csv("titanic_predgbm.csv")